<a href="https://colab.research.google.com/github/sandeepkumar2606/Calculator/blob/master/Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing PySpark

In [1]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=caeb46723188e5255e4b196ba81809c4ec6b12c6cf962fea9a3a2dd008e39a12
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Python Spark regression example")\
        .config("spark.some.config.option", "some-value")\
        .getOrCreate()

In [3]:
!ls

sample_data


In [5]:
df = spark.read.format("csv").\
                options(inferSchema = True, header = True).\
                load("/content/Advertising.csv", header=True)


print(df.show(5, False)) # Shows first five records and does not truncate values

print(df.printSchema())

+---+-----+-----+---------+-----+
|_c0|TV   |Radio|Newspaper|Sales|
+---+-----+-----+---------+-----+
|1  |230.1|37.8 |69.2     |22.1 |
|2  |44.5 |39.3 |45.1     |10.4 |
|3  |17.2 |45.9 |69.3     |9.3  |
|4  |151.5|41.3 |58.5     |18.5 |
|5  |180.8|10.8 |58.4     |12.9 |
+---+-----+-----+---------+-----+
only showing top 5 rows

None
root
 |-- _c0: integer (nullable = true)
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)

None


In [6]:
# Describe the dataset, the result will be shown for all feature columns
df.describe().show()

# Describe the numeriacl statistics for just a signle colums
df.select("TV").describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+
|summary|               _c0|               TV|             Radio|         Newspaper|             Sales|
+-------+------------------+-----------------+------------------+------------------+------------------+
|  count|               200|              200|               200|               200|               200|
|   mean|             100.5|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|57.879184513951124|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|                 1|              0.7|               0.0|               0.3|               1.6|
|    max|               200|            296.4|              49.6|             114.0|              27.0|
+-------+------------------+-----------------+------------------+------------------+------------------+

+-------+-----------------+
|summary|               TV|
+------

## Printing column names

In [7]:
col_names = df.columns
print(col_names)

['_c0', 'TV', 'Radio', 'Newspaper', 'Sales']


## Renaming the column "_c0" to "Index" by using withcolumnRenamed method

In [8]:
df = df.withColumnRenamed("_c0", "Index")

# Printing to show the change in column name
df.show(5)

+-----+-----+-----+---------+-----+
|Index|   TV|Radio|Newspaper|Sales|
+-----+-----+-----+---------+-----+
|    1|230.1| 37.8|     69.2| 22.1|
|    2| 44.5| 39.3|     45.1| 10.4|
|    3| 17.2| 45.9|     69.3|  9.3|
|    4|151.5| 41.3|     58.5| 18.5|
|    5|180.8| 10.8|     58.4| 12.9|
+-----+-----+-----+---------+-----+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula = "Sales ~  TV + Radio + Newspaper")
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show(5, False)

+-----+-----+-----+---------+-----+-----------------+-----+
|Index|TV   |Radio|Newspaper|Sales|features         |label|
+-----+-----+-----+---------+-----+-----------------+-----+
|1    |230.1|37.8 |69.2     |22.1 |[230.1,37.8,69.2]|22.1 |
|2    |44.5 |39.3 |45.1     |10.4 |[44.5,39.3,45.1] |10.4 |
|3    |17.2 |45.9 |69.3     |9.3  |[17.2,45.9,69.3] |9.3  |
|4    |151.5|41.3 |58.5     |18.5 |[151.5,41.3,58.5]|18.5 |
|5    |180.8|10.8 |58.4     |12.9 |[180.8,10.8,58.4]|12.9 |
+-----+-----+-----+---------+-----+-----------------+-----+
only showing top 5 rows



In [10]:
# Same above transformation can be obtained ina sig

testDf = RFormula(formula = "Sales ~  TV + Radio + Newspaper").fit(df).transform(df)

testDf.show(5, False)

+-----+-----+-----+---------+-----+-----------------+-----+
|Index|TV   |Radio|Newspaper|Sales|features         |label|
+-----+-----+-----+---------+-----+-----------------+-----+
|1    |230.1|37.8 |69.2     |22.1 |[230.1,37.8,69.2]|22.1 |
|2    |44.5 |39.3 |45.1     |10.4 |[44.5,39.3,45.1] |10.4 |
|3    |17.2 |45.9 |69.3     |9.3  |[17.2,45.9,69.3] |9.3  |
|4    |151.5|41.3 |58.5     |18.5 |[151.5,41.3,58.5]|18.5 |
|5    |180.8|10.8 |58.4     |12.9 |[180.8,10.8,58.4]|12.9 |
+-----+-----+-----+---------+-----+-----------------+-----+
only showing top 5 rows



# Using Vector Assembler to create Dense Feature Vector

In [11]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler()\
    .setInputCols(["TV","Radio", "Newspaper"])\
    .setOutputCol("va_features")

va.transform(preparedDF).show(5)

+-----+-----+-----+---------+-----+-----------------+-----+-----------------+
|Index|   TV|Radio|Newspaper|Sales|         features|label|      va_features|
+-----+-----+-----+---------+-----+-----------------+-----+-----------------+
|    1|230.1| 37.8|     69.2| 22.1|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
|    2| 44.5| 39.3|     45.1| 10.4| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
|    3| 17.2| 45.9|     69.3|  9.3| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|    4|151.5| 41.3|     58.5| 18.5|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|    5|180.8| 10.8|     58.4| 12.9|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----+-----+-----+---------+-----+-----------------+-----+-----------------+
only showing top 5 rows



# Using RDDs to create feature vector and label

In [12]:
from pyspark.ml.linalg import Vectors

def vecTransform (data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])



vecTransform(df).show(5, False)

+---------------------+-----+
|features             |label|
+---------------------+-----+
|[1.0,230.1,37.8,69.2]|22.1 |
|[2.0,44.5,39.3,45.1] |10.4 |
|[3.0,17.2,45.9,69.3] |9.3  |
|[4.0,151.5,41.3,58.5]|18.5 |
|[5.0,180.8,10.8,58.4]|12.9 |
+---------------------+-----+
only showing top 5 rows



## Using StandardScaler to scale the data

In [13]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler().setInputCol("features").setOutputCol("scaledFeatures")

scaledDF = scaler.fit(preparedDF).transform(preparedDF)

In [ ]:
scaledDF.show(5, False)

+-----+-----+-----+---------+-----+-----------------+-----+----------------------------------------------------------+
|Index|TV   |Radio|Newspaper|Sales|features         |label|scaledFeatures                                            |
+-----+-----+-----+---------+-----+-----------------+-----+----------------------------------------------------------+
|1    |230.1|37.8 |69.2     |22.1 |[230.1,37.8,69.2]|22.1 |[2.6801240087443956,2.546001605562121,3.1774280159006443] |
|2    |44.5 |39.3 |45.1     |10.4 |[44.5,39.3,45.1] |10.4 |[0.5183203754416584,2.6470334153066495,2.0708382011144373]|
|3    |17.2 |45.9 |69.3     |9.3  |[17.2,45.9,69.3] |9.3  |[0.2003395608448657,3.0915733781825754,3.182019674883159] |
|4    |151.5|41.3 |58.5     |18.5 |[151.5,41.3,58.5]|18.5 |[1.7646188062789046,2.781742494966021,2.686120504771498]  |
|5    |180.8|10.8 |58.4     |12.9 |[180.8,10.8,58.4]|12.9 |[2.105894918648356,0.727429030160606,2.6815288457889825]  |
+-----+-----+-----+---------+-----+-------------

In [14]:
preparedDF.show(5)

+-----+-----+-----+---------+-----+-----------------+-----+
|Index|   TV|Radio|Newspaper|Sales|         features|label|
+-----+-----+-----+---------+-----+-----------------+-----+
|    1|230.1| 37.8|     69.2| 22.1|[230.1,37.8,69.2]| 22.1|
|    2| 44.5| 39.3|     45.1| 10.4| [44.5,39.3,45.1]| 10.4|
|    3| 17.2| 45.9|     69.3|  9.3| [17.2,45.9,69.3]|  9.3|
|    4|151.5| 41.3|     58.5| 18.5|[151.5,41.3,58.5]| 18.5|
|    5|180.8| 10.8|     58.4| 12.9|[180.8,10.8,58.4]| 12.9|
+-----+-----+-----+---------+-----+-----------------+-----+
only showing top 5 rows



#Using VectorIndexer to Index feature Vector

In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(preparedDF)

data = featureIndexer.transform(preparedDF)

data.show(5)

+-----+-----+-----+---------+-----+-----------------+-----+-----------------+
|Index|   TV|Radio|Newspaper|Sales|         features|label|  indexedFeatures|
+-----+-----+-----+---------+-----+-----------------+-----+-----------------+
|    1|230.1| 37.8|     69.2| 22.1|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
|    2| 44.5| 39.3|     45.1| 10.4| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
|    3| 17.2| 45.9|     69.3|  9.3| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|    4|151.5| 41.3|     58.5| 18.5|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|    5|180.8| 10.8|     58.4| 12.9|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----+-----+-----+---------+-----+-----------------+-----+-----------------+
only showing top 5 rows



In [16]:
scaler = StandardScaler().setInputCol("indexedFeatures").setOutputCol("scaledFeatures")

finalScaledData = scaler.fit(data).transform(data)

finalScaledData.show(5, False)

+-----+-----+-----+---------+-----+-----------------+-----+-----------------+----------------------------------------------------------+
|Index|TV   |Radio|Newspaper|Sales|features         |label|indexedFeatures  |scaledFeatures                                            |
+-----+-----+-----+---------+-----+-----------------+-----+-----------------+----------------------------------------------------------+
|1    |230.1|37.8 |69.2     |22.1 |[230.1,37.8,69.2]|22.1 |[230.1,37.8,69.2]|[2.6801240087443956,2.546001605562121,3.1774280159006443] |
|2    |44.5 |39.3 |45.1     |10.4 |[44.5,39.3,45.1] |10.4 |[44.5,39.3,45.1] |[0.5183203754416584,2.6470334153066495,2.0708382011144373]|
|3    |17.2 |45.9 |69.3     |9.3  |[17.2,45.9,69.3] |9.3  |[17.2,45.9,69.3] |[0.2003395608448657,3.0915733781825754,3.182019674883159] |
|4    |151.5|41.3 |58.5     |18.5 |[151.5,41.3,58.5]|18.5 |[151.5,41.3,58.5]|[1.7646188062789046,2.781742494966021,2.686120504771498]  |
|5    |180.8|10.8 |58.4     |12.9 |[180.8

In [17]:
inputData = finalScaledData.select(["scaledFeatures", "label"])

# inputData = inputData.withColumnRenamed("scaledFeatures", "features")

inputData.show(5, False)

+----------------------------------------------------------+-----+
|scaledFeatures                                            |label|
+----------------------------------------------------------+-----+
|[2.6801240087443956,2.546001605562121,3.1774280159006443] |22.1 |
|[0.5183203754416584,2.6470334153066495,2.0708382011144373]|10.4 |
|[0.2003395608448657,3.0915733781825754,3.182019674883159] |9.3  |
|[1.7646188062789046,2.781742494966021,2.686120504771498]  |18.5 |
|[2.105894918648356,0.727429030160606,2.6815288457889825]  |12.9 |
+----------------------------------------------------------+-----+
only showing top 5 rows



# Fitting a Linear Regression Model

In [18]:
training, test = inputData.randomSplit([0.8,0.2])

print("Training set has {} records".format(training.count()))
print("Training set has {} records".format(test.count()))

Training set has 169 records
Training set has 31 records


# Creating an instace of LinearRegression class passing on parameters

In [19]:
lin_reg = LinearRegression(featuresCol="scaledFeatures", labelCol = "label").setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

print(lin_reg.explainParams()) # Explains the parameters of the model

model = lin_reg.fit(training) # Fitting the Model

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.8)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features, current: scaledFeatures)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter:

In [20]:
print('###' * 30)
print("Coffiecients of the model are: {}".format(model.coefficients))
print('###' * 30)
print('###' * 30)

print("Intercept of linear regression equation is: {}".format(model.intercept))

print('###' * 30)
print('###' * 30)
summary = model.summary # creating an instance of model summary

print('###' * 30)
print('###' * 30)
print("The residual values after the fit are: ")
summary.residuals.show() # This returns only the first 20 values

print('###' * 30)
print('###' * 30)
print("The calculated loss for the objective function at each iteration is: ")
print(summary.objectiveHistory)

print('###' * 30)
print('###' * 30)
print("The RMSE value of the fitted model: {}".format(summary.rootMeanSquaredError))

print('###' * 30)
print('###' * 30)
print("The R2 score of the fitted model: {}".format(summary.r2))

##########################################################################################
Coffiecients of the model are: [3.5935141589659745,2.578221991929001,0.0]
##########################################################################################
##########################################################################################
Intercept of linear regression equation is: 3.8368710089065368
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################
The residual values after the fit are: 
+-------------------+
|          residuals|
+-------------------+
| -9.142906454208802|
| -2.822878060596243|
|  -3.95517654111175|
|-3.2118761339049025|
|0.23849205957011588|
|-3.434453

# Predictions on Test Data

In [21]:
predictions = model.transform(test)
predictions.show(30, False)

+-----------------------------------------------------------+-----+------------------+
|scaledFeatures                                             |label|prediction        |
+-----------------------------------------------------------+-----+------------------+
|[0.08502783687020464,1.8926625692141694,1.9009468187613678]|5.5  |9.022124003854682 |
|[0.09085166131336933,2.620091599374775,2.323379445152783]  |6.6  |10.918525522578184|
|[0.1013345453110658,3.2936369976716326,3.443744236886536]  |7.2  |12.69274547310224 |
|[0.44261065768051727,2.7143879551363352,0.5464074189193304]|10.9 |12.425693394750407|
|[0.5008489021121643,1.7444825815888607,0.9412900914156532] |9.6  |10.13434198653864 |
|[0.7256485256183217,0.8486672018540403,0.8402735938003149] |9.7  |8.632551703797695 |
|[0.779227710495437,0.7880481160073232,1.6897305055656602]  |9.7  |8.668799803018826 |
|[0.8700793718088063,3.3273142675864755,2.0983881550095296] |14.7 |15.54206836967629 |
|[1.0284873966628862,1.7175407656569863,3.3

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

reg_eval = RegressionEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "rmse")

print("The RMSE value on test data: {}".format(reg_eval.evaluate(predictions)))

The RMSE value on test data: 1.914127303602714
